In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd CS230

/content/gdrive/My Drive/CS230


In [0]:
! git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 26970 (delta 1), reused 2 (delta 0), pack-reused 26963
Receiving objects: 100% (26970/26970), 25.18 MiB | 4.25 MiB/s, done.
Resolving deltas: 100% (18753/18753), done.
Checking out files: 100% (700/700), done.


In [0]:
! pip install tokenizers

     |████████████████████████████████| 3.8MB 2.8MB/s 


In [0]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    # Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

In [0]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=50000, show_progress=True, initial_alphabet=ByteLevel.alphabet(), special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.train(trainer, ["all.txt"])

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 50000


In [0]:
# You will see the generated files in the output.
tokenizer.model.save('.')

['./vocab.json', './merges.txt']

In [0]:
# Let's tokenizer a simple input
tokenizer.model = BPE('vocab.json', 'merges.txt')
encoding = tokenizer.encode("This is a Simple INPUT to be Tokenized")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['ĠThis', 'Ġis', 'Ġa', 'ĠSimple', 'ĠIN', 'PUT', 'Ġto', 'Ġbe', 'ĠTo', 'ken', 'ized']
Decoded string:  This is a Simple INPUT to be Tokenized


In [0]:
!pip install transformers
!pip install tensorflow==2.1.0

In [0]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "vocab.json",
    "merges.txt",
)

In [0]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [0]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(".", max_len=512)

In [0]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)

CPU times: user 8.78 s, sys: 370 ms, total: 9.15 s
Wall time: 6 s


In [0]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [0]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [0]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERTtest",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
%%time
trainer.train()

{"loss": 5.592786348342895, "learning_rate": 4.518953242255147e-05, "epoch": 0.09620935154897056, "step": 500}
{"loss": 4.825703997611999, "learning_rate": 4.037906484510295e-05, "epoch": 0.19241870309794112, "step": 1000}
{"loss": 4.525733775615692, "learning_rate": 3.556859726765442e-05, "epoch": 0.28862805464691166, "step": 1500}
{"loss": 4.362973469257355, "learning_rate": 3.075812969020589e-05, "epoch": 0.38483740619588225, "step": 2000}
{"loss": 4.205488008022308, "learning_rate": 2.594766211275736e-05, "epoch": 0.4810467577448528, "step": 2500}
{"loss": 4.153908041954041, "learning_rate": 2.1137194535308832e-05, "epoch": 0.5772561092938233, "step": 3000}
{"loss": 4.080144800186157, "learning_rate": 1.6326726957860305e-05, "epoch": 0.6734654608427939, "step": 3500}
{"loss": 3.9908780226707457, "learning_rate": 1.1516259380411776e-05, "epoch": 0.7696748123917645, "step": 4000}
{"loss": 3.969428906917572, "learning_rate": 6.705791802963249e-06, "epoch": 0.8658841639407351, "step": 

TrainOutput(global_step=5197, training_loss=4.346814475289808)

In [0]:
trainer.save_model("./BERTtest")

In [0]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERTtest",
    tokenizer=tokenizer
)

In [0]:
fill_mask("I love <mask>.")

[{'score': 0.06602732837200165,
  'sequence': '<s> I love you.</s>',
  'token': 300},
 {'score': 0.056926608085632324,
  'sequence': '<s> I love a.</s>',
  'token': 264},
 {'score': 0.047980278730392456,
  'sequence': '<s> I love to.</s>',
  'token': 286},
 {'score': 0.032504189759492874,
  'sequence': '<s> I love the.</s>',
  'token': 268},
 {'score': 0.0323076993227005, 'sequence': '<s> I love it.</s>', 'token': 339}]

In [0]:
model = RobertaForMaskedLM.from_pretrained("./BERTtest")

In [0]:
tokenizer = RobertaTokenizerFast.from_pretrained(".", max_len=512)


In [0]:
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

In [0]:
!pip install -q tensorflow==2.1

In [0]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [0]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

AttributeError: ignored

In [0]:
input_ids.dim()

AttributeError: ignored

In [0]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

AttributeError: ignored

In [0]:
from transformers import TransfoXLTokenizerFast
tokenizer = TransfoXLTokenizerFast.from_pretrained(".", max_len=512, pad_token_id=tokenizer.eos_token_id)

In [0]:
encoding = tokenizer.encode("This is a Simple INPUT to be Tokenized")

Using pad_token, but it is not set yet.


In [0]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)

Using pad_token, but it is not set yet.


CPU times: user 4.46 s, sys: 9.32 ms, total: 4.47 s
Wall time: 3.57 s


In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Trying to run TransfoXL on all.txt

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/CS230

/content/gdrive/My Drive/CS230


In [0]:
!pip install tokenizers
!pip install transformers
!pip install tensorflow==2.1.0

     |████████████████████████████████| 3.8MB 5.9MB/s 
     |████████████████████████████████| 665kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 16.2MB/s 
     |████████████████████████████████| 890kB 28.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=113d06ad6df651ffb10b0c24cbaa8a933c472fbe269dbd028839618c9899be54
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 421.8MB 18kB/s 
     |████████████████████████████████| 3.9MB 36.2MB/s 
     |████████████████████████████████| 450kB 39.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b717c90ed137f2f102964f54434792c662baf8cceca7970959163727c719d521
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.

In [0]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer, ByteLevelBPETokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = ByteLevelBPETokenizer()

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    # Lowercase()
])

# # Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
# tokenizer.pre_tokenizer = ByteLevel()

# # And finally, let's plug a decoder so we can recover from a tokenized input to the original one
# tokenizer.decoder = ByteLevelDecoder()

In [0]:

# Customize training
tokenizer.train(files="all.txt", vocab_size=100_000, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 82363


In [0]:
# from tokenizers.trainers import BpeTrainer

# # We initialize our trainer, giving him the details about the vocabulary we want to generate
# trainer = BpeTrainer(vocab_size=100000, show_progress=True, initial_alphabet=ByteLevel.alphabet(), special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])
# tokenizer.train(trainer, ["all.txt"])

# print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 68848


In [0]:
# You will see the generated files in the output.
tokenizer.save('.')

['./vocab.json', './merges.txt']

In [0]:
from tokenizers.processors import BertProcessing

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [0]:
tokenizer.encode("Mi estas Julien.")

[14726, 24, 24]

In [0]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mefpid8x
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-mefpid8x
     |████████████████████████████████| 3.8MB 9.1MB/s 
     |████████████████████████████████| 1.1MB 56.8MB/s 
     |████████████████████████████████| 890kB 58.1MB/s 
  Created wheel for transformers: filename=transformers-2.10.0-cp36-none-any.whl size=654737 sha256=f61eb05d29e6adae2ec2e5cfa5574f425170aeced0847871e815e7a4570658f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-nh3fj7cf/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=65080445500157db076ede27609e07c6443bbe104a5618969bc8178db4c7115c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses
tokenizers               0.7.0          
tr

In [0]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [0]:
from transformers import TransfoXLTokenizerFast
tokenizer = TransfoXLTokenizerFast.from_pretrained('.')

In [0]:
tokenizer.pad_token = '<pad>'
tokenizer.bos_token = '<s>'
tokenizer.unk_token = '<unk>'

In [0]:
len(tokenizer)

82363

In [0]:
from transformers import TransfoXLConfig

config = TransfoXLConfig(
    n_head=8,
    n_layer=9
)

In [0]:
from transformers import TransfoXLLMHeadModel

model = TransfoXLLMHeadModel(config=config)

In [0]:
model.num_parameters()

138818378

In [0]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)

CPU times: user 3.61 s, sys: 60.4 ms, total: 3.67 s
Wall time: 3 s


In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
# %%time
trainer.train()

RuntimeError: ignored

In [0]:
trainer.save_model(".")

# Next attempt

In [0]:
! git clone https://github.com/kimiyoung/transformer-xl.git

Cloning into 'transformer-xl'...
remote: Enumerating objects: 164, done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (164/164), 113.49 KiB | 210.00 KiB/s, done.
Resolving deltas: 100% (98/98), done.


In [0]:
%cd transformer-xl

/content/transformer-xl


In [0]:
! pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [0]:
%cd pytorch

/content/transformer-xl/pytorch


In [0]:
! ls

data_utils.py	    run_enwik8_base.sh	 run_text8_base.sh   train.py
eval.py		    run_enwik8_large.sh  run_text8_large.sh  utils
mem_transformer.py  run_lm1b_base.sh	 run_wt103_base.sh
README.md	    run_lm1b_large.sh	 run_wt103_large.sh


In [0]:
 !   python train.py \
        --cuda \
        --data ../data/one-billion-words/ \
        --dataset lm1b \
        --adaptive \
        --n_layer 18 \
        --d_model 1024 \
        --div_val 4 \
        --n_head 8 \
        --d_head 128 \
        --d_inner 4096 \
        --dropout 0.0 \
        --dropatt 0.0 \
        --optim adam \
        --warmup_step 20000 \
        --max_step 500000 \
        --lr 0.00025 \
        --tgt_len 32 \
        --mem_len 32 \
        --eval_tgt_len 32 \
        --batch_size 224 \
        --multi_gpu \
        --gpu0_bsz 32 \

Experiment dir : LM-TFM-lm1b/20200525-213852
Producing dataset lm1b...
building vocab from ../data/one-billion-words/1b_word_vocab.txt
Traceback (most recent call last):
  File "train.py", line 184, in <module>
    corpus = get_lm_corpus(args.data, args.dataset)
  File "/content/transformer-xl/pytorch/data_utils.py", line 257, in get_lm_corpus
    corpus = Corpus(datadir, dataset, **kwargs)
  File "/content/transformer-xl/pytorch/data_utils.py", line 196, in __init__
    self.vocab.build_vocab()
  File "/content/transformer-xl/pytorch/utils/vocabulary.py", line 74, in build_vocab
    self._build_from_file(self.vocab_file)
  File "/content/transformer-xl/pytorch/utils/vocabulary.py", line 65, in _build_from_file
    with open(vocab_file, 'r', encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: '../data/one-billion-words/1b_word_vocab.txt'


# Fine-tuning GPT-2

In [1]:
pip install transformers

     |████████████████████████████████| 665kB 3.4MB/s 
     |████████████████████████████████| 3.8MB 18.1MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5d943deb13a33044d0babf0b0a5d4a186cfc833070424583c608977079d43a00
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2TokenizerFast
import torch

config = GPT2Config()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.add_tokens('<pad>')
tokenizer.pad_token = '<pad>'

model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)



In [0]:
model = GPT2LMHeadModel(config=config)

In [6]:
len(tokenizer)

50258

In [0]:
# %%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)

In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./GPT2-2",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
trainer.train()

{"loss": 1.6411301233172417, "learning_rate": 4.879738310563787e-05, "epoch": 0.04810467577448528, "step": 500}
{"loss": 1.5126444403529167, "learning_rate": 4.759476621127574e-05, "epoch": 0.09620935154897056, "step": 1000}
{"loss": 1.435596613585949, "learning_rate": 4.639214931691361e-05, "epoch": 0.14431402732345583, "step": 1500}
{"loss": 1.4430792790055276, "learning_rate": 4.518953242255147e-05, "epoch": 0.19241870309794112, "step": 2000}
{"loss": 1.408036316394806, "learning_rate": 4.398691552818934e-05, "epoch": 0.2405233788724264, "step": 2500}
{"loss": 1.40939171564579, "learning_rate": 4.278429863382721e-05, "epoch": 0.28862805464691166, "step": 3000}
{"loss": 1.3941316257715226, "learning_rate": 4.158168173946508e-05, "epoch": 0.33673273042139695, "step": 3500}
{"loss": 1.3874358997941016, "learning_rate": 4.037906484510295e-05, "epoch": 0.38483740619588225, "step": 4000}
{"loss": 1.3601308676600457, "learning_rate": 3.917644795074081e-05, "epoch": 0.43294208197036754, "st

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 1.3186585757434368, "learning_rate": 2.474504521839523e-05, "epoch": 1.0101981912641909, "step": 10500}
{"loss": 1.2645536024570465, "learning_rate": 2.3542428324033098e-05, "epoch": 1.0583028670386763, "step": 11000}
{"loss": 1.2532881612181663, "learning_rate": 2.2339811429670963e-05, "epoch": 1.1064075428131614, "step": 11500}
{"loss": 1.248851757645607, "learning_rate": 2.1137194535308832e-05, "epoch": 1.1545122185876466, "step": 12000}
{"loss": 1.264731287062168, "learning_rate": 1.99345776409467e-05, "epoch": 1.202616894362132, "step": 12500}
{"loss": 1.2975292428135872, "learning_rate": 1.8731960746584567e-05, "epoch": 1.2507215701366172, "step": 13000}
{"loss": 1.2829569338262081, "learning_rate": 1.7529343852222436e-05, "epoch": 1.2988262459111026, "step": 13500}
{"loss": 1.2744102444052696, "learning_rate": 1.6326726957860305e-05, "epoch": 1.3469309216855878, "step": 14000}
{"loss": 1.3006766911149026, "learning_rate": 1.5124110063498173e-05, "epoch": 1.3950355974600

TrainOutput(global_step=20788, training_loss=1.3291308207665702)

In [0]:
trainer.save_model("./GPT2-2")

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./GPT2-2-4",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [8]:
trainer.train()

{"loss": 1.64058141040802, "learning_rate": 4.975947662112758e-05, "epoch": 0.04810467577448528, "step": 500}
{"loss": 1.5136681725382806, "learning_rate": 4.951895324225515e-05, "epoch": 0.09620935154897056, "step": 1000}
{"loss": 1.4370932877063751, "learning_rate": 4.927842986338273e-05, "epoch": 0.14431402732345583, "step": 1500}
{"loss": 1.4444992812871933, "learning_rate": 4.9037906484510296e-05, "epoch": 0.19241870309794112, "step": 2000}
{"loss": 1.4099103708863259, "learning_rate": 4.879738310563787e-05, "epoch": 0.2405233788724264, "step": 2500}
{"loss": 1.4115998622775079, "learning_rate": 4.855685972676544e-05, "epoch": 0.28862805464691166, "step": 3000}
{"loss": 1.3949111145734787, "learning_rate": 4.8316336347893015e-05, "epoch": 0.33673273042139695, "step": 3500}
{"loss": 1.388596108198166, "learning_rate": 4.807581296902059e-05, "epoch": 0.38483740619588225, "step": 4000}
{"loss": 1.3618152047991752, "learning_rate": 4.7835289590148165e-05, "epoch": 0.43294208197036754,

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 1.3205280587077142, "learning_rate": 4.494900904367905e-05, "epoch": 1.0101981912641909, "step": 10500}
{"loss": 1.2576406440734864, "learning_rate": 4.470848566480662e-05, "epoch": 1.0583028670386763, "step": 11000}
{"loss": 1.2505618564486503, "learning_rate": 4.44679622859342e-05, "epoch": 1.1064075428131614, "step": 11500}
{"loss": 1.2458334935307502, "learning_rate": 4.4227438907061766e-05, "epoch": 1.1545122185876466, "step": 12000}
{"loss": 1.260573048532009, "learning_rate": 4.398691552818934e-05, "epoch": 1.202616894362132, "step": 12500}
{"loss": 1.2955674206018448, "learning_rate": 4.3746392149316916e-05, "epoch": 1.2507215701366172, "step": 13000}
{"loss": 1.2796441693902016, "learning_rate": 4.350586877044449e-05, "epoch": 1.2988262459111026, "step": 13500}
{"loss": 1.2713103578090668, "learning_rate": 4.3265345391572066e-05, "epoch": 1.3469309216855878, "step": 14000}
{"loss": 1.2984353561401367, "learning_rate": 4.3024822012699635e-05, "epoch": 1.395035597460073

{"loss": 1.2144421777129173, "learning_rate": 3.989801808735809e-05, "epoch": 2.0203963825283817, "step": 21000}
{"loss": 1.181298919916153, "learning_rate": 3.965749470848567e-05, "epoch": 2.068501058302867, "step": 21500}
{"loss": 1.1952700661420823, "learning_rate": 3.941697132961324e-05, "epoch": 2.1166057340773525, "step": 22000}
{"loss": 1.1856752136349678, "learning_rate": 3.917644795074081e-05, "epoch": 2.1647104098518377, "step": 22500}
{"loss": 1.1954678372442722, "learning_rate": 3.8935924571868385e-05, "epoch": 2.212815085626323, "step": 23000}
{"loss": 1.187569848895073, "learning_rate": 3.869540119299596e-05, "epoch": 2.260919761400808, "step": 23500}
{"loss": 1.2202401177883149, "learning_rate": 3.8454877814123536e-05, "epoch": 2.3090244371752933, "step": 24000}
{"loss": 1.2189407121539115, "learning_rate": 3.821435443525111e-05, "epoch": 2.357129112949779, "step": 24500}
{"loss": 1.227940095305443, "learning_rate": 3.7973831056378686e-05, "epoch": 2.405233788724264, "st

{"loss": 1.162742212831974, "learning_rate": 3.484702713103714e-05, "epoch": 3.030594573792573, "step": 31500}
{"loss": 1.136767702996731, "learning_rate": 3.460650375216471e-05, "epoch": 3.078699249567058, "step": 32000}
{"loss": 1.1492107038497925, "learning_rate": 3.4365980373292286e-05, "epoch": 3.126803925341543, "step": 32500}
{"loss": 1.1060473197996616, "learning_rate": 3.4125456994419855e-05, "epoch": 3.1749086011160284, "step": 33000}
{"loss": 1.1616924324035645, "learning_rate": 3.388493361554743e-05, "epoch": 3.223013276890514, "step": 33500}
{"loss": 1.1639852101802826, "learning_rate": 3.3644410236675005e-05, "epoch": 3.271117952664999, "step": 34000}
{"loss": 1.1524310979545116, "learning_rate": 3.340388685780258e-05, "epoch": 3.3192226284394843, "step": 34500}
{"loss": 1.1723134739995003, "learning_rate": 3.3163363478930156e-05, "epoch": 3.3673273042139695, "step": 35000}
{"loss": 1.1690775940418243, "learning_rate": 3.292284010005773e-05, "epoch": 3.4154319799884547, "

...

{"loss": 1.0872215096354485, "learning_rate": 1.99345776409467e-05, "epoch": 6.01308447181066, "step": 62500}
{"loss": 1.0345839071571827, "learning_rate": 1.9694054262074273e-05, "epoch": 6.061189147585146, "step": 63000}
{"loss": 1.0572419431209563, "learning_rate": 1.945353088320185e-05, "epoch": 6.10929382335963, "step": 63500}
{"loss": 1.0517034419476985, "learning_rate": 1.9213007504329424e-05, "epoch": 6.157398499134116, "step": 64000}
{"loss": 1.0645298295319081, "learning_rate": 1.8972484125456995e-05, "epoch": 6.205503174908601, "step": 64500}
{"loss": 1.0582401767969132, "learning_rate": 1.8731960746584567e-05, "epoch": 6.253607850683086, "step": 65000}
{"loss": 1.046628348827362, "learning_rate": 1.8491437367712142e-05, "epoch": 6.301712526457572, "step": 65500}
{"loss": 1.057432470560074, "learning_rate": 1.8250913988839714e-05, "epoch": 6.349817202232057, "step": 66000}
{"loss": 1.07873018810153, "learning_rate": 1.801039060996729e-05, "epoch": 6.397921878006542, "step": 

{"loss": 1.0558339683115483, "learning_rate": 1.4883586684625748e-05, "epoch": 7.023282663074851, "step": 73000}
{"loss": 1.0317472827136516, "learning_rate": 1.4643063305753321e-05, "epoch": 7.071387338849336, "step": 73500}
{"loss": 1.0553042123615741, "learning_rate": 1.4402539926880893e-05, "epoch": 7.119492014623821, "step": 74000}
{"loss": 1.0200629687011242, "learning_rate": 1.4162016548008467e-05, "epoch": 7.167596690398307, "step": 74500}
{"loss": 1.0458686141073703, "learning_rate": 1.392149316913604e-05, "epoch": 7.215701366172792, "step": 75000}
{"loss": 1.0605863770842552, "learning_rate": 1.3680969790263615e-05, "epoch": 7.263806041947277, "step": 75500}
{"loss": 1.0368460617959498, "learning_rate": 1.3440446411391189e-05, "epoch": 7.311910717721762, "step": 76000}
{"loss": 1.0417096337676048, "learning_rate": 1.3199923032518762e-05, "epoch": 7.360015393496248, "step": 76500}
{"loss": 1.0335262971818446, "learning_rate": 1.2959399653646334e-05, "epoch": 7.408120069270733,

{"loss": 1.0099513162374496, "learning_rate": 9.83259572830479e-06, "epoch": 8.033480854339041, "step": 83500}
{"loss": 1.0175067630708217, "learning_rate": 9.592072349432366e-06, "epoch": 8.081585530113527, "step": 84000}
{"loss": 1.003861258596182, "learning_rate": 9.351548970559938e-06, "epoch": 8.129690205888012, "step": 84500}
{"loss": 0.977281278938055, "learning_rate": 9.111025591687513e-06, "epoch": 8.177794881662498, "step": 85000}
{"loss": 1.0333355256915093, "learning_rate": 8.870502212815086e-06, "epoch": 8.225899557436982, "step": 85500}
{"loss": 1.03045860221982, "learning_rate": 8.62997883394266e-06, "epoch": 8.274004233211468, "step": 86000}
{"loss": 1.0165567412376404, "learning_rate": 8.389455455070233e-06, "epoch": 8.322108908985953, "step": 86500}
{"loss": 1.0164050607979298, "learning_rate": 8.148932076197807e-06, "epoch": 8.370213584760439, "step": 87000}
{"loss": 1.0372441437244415, "learning_rate": 7.90840869732538e-06, "epoch": 8.418318260534924, "step": 87500}

{"loss": 1.020999424368143, "learning_rate": 4.781604771983837e-06, "epoch": 9.043679045603232, "step": 94000}
{"loss": 0.998709222316742, "learning_rate": 4.5410813931114104e-06, "epoch": 9.091783721377718, "step": 94500}
{"loss": 0.9899536942541599, "learning_rate": 4.300558014238984e-06, "epoch": 9.139888397152204, "step": 95000}
{"loss": 1.013422545224428, "learning_rate": 4.060034635366558e-06, "epoch": 9.18799307292669, "step": 95500}
{"loss": 1.0115390468537808, "learning_rate": 3.819511256494132e-06, "epoch": 9.236097748701173, "step": 96000}
{"loss": 1.0001398680210114, "learning_rate": 3.5789878776217047e-06, "epoch": 9.284202424475659, "step": 96500}
{"loss": 1.0066975376605987, "learning_rate": 3.338464498749279e-06, "epoch": 9.332307100250144, "step": 97000}
{"loss": 1.0146661387383937, "learning_rate": 3.097941119876852e-06, "epoch": 9.38041177602463, "step": 97500}
{"loss": 1.0064007141888143, "learning_rate": 2.8574177410044256e-06, "epoch": 9.428516451799116, "step": 9

TrainOutput(global_step=103940, training_loss=1.1379486533073604)

In [0]:
trainer.save_model("./GPT2-2-4")

In [0]:
!pip install -q tensorflow==2.1

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 3.9MB 59.1MB/s 
     |████████████████████████████████| 450kB 58.5MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [0]:
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

In [0]:
print(input_ids)

tensor([[   40,  2883,  6155,   351,   616, 13779,  3290]])


In [0]:
greedy_output = model.generate(input_ids, max_length=50)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


RuntimeError: ignored

# GPT2 Generation

In [4]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2TokenizerFast
import torch

config = GPT2Config()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.add_tokens('<pad>')
tokenizer.pad_token = '<pad>'

model = GPT2LMHeadModel.from_pretrained('./GPT2-2-4', config=config)



In [19]:
prompt = '''JOHNNY
Well because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.

MARK
So, I mean, what's the interesting part?

JOHNNY
Well the interesting part is that
'''
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
outputs = model.generate(inputs, max_length=300, do_sample=True, top_p=0.95, top_k=100, temperature=1.1)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [20]:
tokenizer.decode(outputs[0].numpy())

"JOHNNY\nWell because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.\n\nMARK\nSo, I mean, what's the interesting part?\n\nJOHNNY\nWell the interesting part is that\n! Our thing! John manages to take a couple's arm, takes it off, and- oh wellJohn! We can hear John over and under the arm, and they have to separate. John and the arm and he tries to take his free arm with a great big enough to jerk the OTHER MAN away. It's in too big to lift, John, but in the driver doesn't. John is also gets out so he can take back the free arm, lifts John too much too. At the very back and we hear. He grabs his free arm - to catch Marley's head and lifts up to stop him, as much as he can. They come - there! John and, John can just'see it, there. She ykes him over and Marley, so high he can see his.av - he's practically jumps! He rides on his head, running. John lifts the

In [0]:
with open("Room-2.txt", "w") as f:
  f.write(tokenizer.decode(outputs[0].numpy()))

In [0]:
tokenizer.decode([13909, 3069, 46])

'HELLO'

# GPT2 only scripts

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/My Drive/CS230

/content/gdrive/My Drive/CS230


In [0]:
pip install transformers

     |████████████████████████████████| 665kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 44.1MB/s 
     |████████████████████████████████| 890kB 38.1MB/s 
     |████████████████████████████████| 3.8MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1e256382de12f4ef7b31edaec849ebd8b250496cd09b1448c374766b612b9858
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2TokenizerFast
import torch

config = GPT2Config()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', max_len=512)
tokenizer.pad_token = '<pad>'

model = GPT2LMHeadModel.from_pretrained('./GPT2new/checkpoint-40000', config=config)

In [0]:
# %%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)

In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./GPT2new-2",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
trainer.train()

{"loss": 3.153368466377258, "learning_rate": 4.9398662625679516e-05, "epoch": 0.024053494972819552, "step": 500}
{"loss": 2.707065662205219, "learning_rate": 4.879732525135903e-05, "epoch": 0.048106989945639105, "step": 1000}
{"loss": 2.5705057479143143, "learning_rate": 4.8195987877038536e-05, "epoch": 0.07216048491845865, "step": 1500}
{"loss": 2.478780880153179, "learning_rate": 4.759465050271805e-05, "epoch": 0.09621397989127821, "step": 2000}
{"loss": 2.323019295334816, "learning_rate": 4.699331312839756e-05, "epoch": 0.12026747486409775, "step": 2500}
{"loss": 2.327748654127121, "learning_rate": 4.639197575407707e-05, "epoch": 0.1443209698369173, "step": 3000}
{"loss": 2.31366570019722, "learning_rate": 4.5790638379756575e-05, "epoch": 0.16837446480973686, "step": 3500}
{"loss": 2.1833542249798774, "learning_rate": 4.518930100543609e-05, "epoch": 0.19242795978255642, "step": 4000}
{"loss": 2.1661865586042404, "learning_rate": 4.45879636311156e-05, "epoch": 0.21648145475537595, "s

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 2.0489576862454415, "learning_rate": 3.7371915139269735e-05, "epoch": 0.5051233944292106, "step": 10500}
{"loss": 2.0135043615698813, "learning_rate": 3.677057776494925e-05, "epoch": 0.5291768894020301, "step": 11000}
{"loss": 1.9867573385238648, "learning_rate": 3.616924039062876e-05, "epoch": 0.5532303843748496, "step": 11500}
{"loss": 1.9814091719985008, "learning_rate": 3.5567903016308275e-05, "epoch": 0.5772838793476692, "step": 12000}
{"loss": 1.9895299215912818, "learning_rate": 3.496656564198778e-05, "epoch": 0.6013373743204887, "step": 12500}
{"loss": 1.9607399579286575, "learning_rate": 3.4365228267667294e-05, "epoch": 0.6253908692933083, "step": 13000}
{"loss": 1.970361325442791, "learning_rate": 3.376389089334681e-05, "epoch": 0.6494443642661278, "step": 13500}
{"loss": 1.9729110438227653, "learning_rate": 3.316255351902632e-05, "epoch": 0.6734978592389474, "step": 14000}
{"loss": 1.936430246412754, "learning_rate": 3.256121614470583e-05, "epoch": 0.697551354211767

{"loss": 1.8605616526007653, "learning_rate": 2.4743830278539474e-05, "epoch": 1.0102467888584212, "step": 21000}
{"loss": 1.8424137750267982, "learning_rate": 2.4142492904218984e-05, "epoch": 1.0343002838312407, "step": 21500}
{"loss": 1.8258581936955451, "learning_rate": 2.3541155529898497e-05, "epoch": 1.0583537788040602, "step": 22000}
{"loss": 1.7995699836611747, "learning_rate": 2.2939818155578007e-05, "epoch": 1.0824072737768797, "step": 22500}
{"loss": 1.80790741366148, "learning_rate": 2.2338480781257517e-05, "epoch": 1.1064607687496992, "step": 23000}
{"loss": 1.8184301874637603, "learning_rate": 2.173714340693703e-05, "epoch": 1.130514263722519, "step": 23500}
{"loss": 1.7771041305661202, "learning_rate": 2.113580603261654e-05, "epoch": 1.1545677586953385, "step": 24000}
{"loss": 1.813562238574028, "learning_rate": 2.0534468658296053e-05, "epoch": 1.178621253668158, "step": 24500}
{"loss": 1.8011977238059045, "learning_rate": 1.9933131283975563e-05, "epoch": 1.20267474864097

TrainOutput(global_step=41574, training_loss=1.9438347338421518)

In [0]:
trainer.save_model("./GPT2new")

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./GPT2new-2",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
trainer.train()

{"loss": 1.5512563256025313, "learning_rate": 4.975947662112758e-05, "epoch": 0.04810467577448528, "step": 500}
{"loss": 1.5930414560437203, "learning_rate": 4.951895324225515e-05, "epoch": 0.09620935154897056, "step": 1000}
{"loss": 1.549421107530594, "learning_rate": 4.927842986338273e-05, "epoch": 0.14431402732345583, "step": 1500}
{"loss": 1.5732636432051659, "learning_rate": 4.9037906484510296e-05, "epoch": 0.19241870309794112, "step": 2000}
{"loss": 1.5443210139870645, "learning_rate": 4.879738310563787e-05, "epoch": 0.2405233788724264, "step": 2500}
{"loss": 1.5555437121391296, "learning_rate": 4.855685972676544e-05, "epoch": 0.28862805464691166, "step": 3000}
{"loss": 1.5414656393527986, "learning_rate": 4.8316336347893015e-05, "epoch": 0.33673273042139695, "step": 3500}
{"loss": 1.5455627790689468, "learning_rate": 4.807581296902059e-05, "epoch": 0.38483740619588225, "step": 4000}
{"loss": 1.5174515696763993, "learning_rate": 4.7835289590148165e-05, "epoch": 0.4329420819703675

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 1.4718061773777007, "learning_rate": 4.494900904367905e-05, "epoch": 1.0101981912641909, "step": 10500}
{"loss": 1.4183730962872505, "learning_rate": 4.470848566480662e-05, "epoch": 1.0583028670386763, "step": 11000}
{"loss": 1.3997100524306296, "learning_rate": 4.44679622859342e-05, "epoch": 1.1064075428131614, "step": 11500}
{"loss": 1.3980496806502343, "learning_rate": 4.4227438907061766e-05, "epoch": 1.1545122185876466, "step": 12000}
{"loss": 1.419847181379795, "learning_rate": 4.398691552818934e-05, "epoch": 1.202616894362132, "step": 12500}
{"loss": 1.4515884473323821, "learning_rate": 4.3746392149316916e-05, "epoch": 1.2507215701366172, "step": 13000}
{"loss": 1.438564662516117, "learning_rate": 4.350586877044449e-05, "epoch": 1.2988262459111026, "step": 13500}
{"loss": 1.4327191655039788, "learning_rate": 4.3265345391572066e-05, "epoch": 1.3469309216855878, "step": 14000}
{"loss": 1.458894559085369, "learning_rate": 4.3024822012699635e-05, "epoch": 1.395035597460073, 

{"loss": 1.3808122606277466, "learning_rate": 3.989801808735809e-05, "epoch": 2.0203963825283817, "step": 21000}
{"loss": 1.3243235739469528, "learning_rate": 3.965749470848567e-05, "epoch": 2.068501058302867, "step": 21500}
{"loss": 1.3466491847634317, "learning_rate": 3.941697132961324e-05, "epoch": 2.1166057340773525, "step": 22000}
{"loss": 1.3370804866254329, "learning_rate": 3.917644795074081e-05, "epoch": 2.1647104098518377, "step": 22500}
{"loss": 1.345180025756359, "learning_rate": 3.8935924571868385e-05, "epoch": 2.212815085626323, "step": 23000}
{"loss": 1.3446701183319092, "learning_rate": 3.869540119299596e-05, "epoch": 2.260919761400808, "step": 23500}
{"loss": 1.368750080883503, "learning_rate": 3.8454877814123536e-05, "epoch": 2.3090244371752933, "step": 24000}
{"loss": 1.3729849476218223, "learning_rate": 3.821435443525111e-05, "epoch": 2.357129112949779, "step": 24500}
{"loss": 1.3818097634315492, "learning_rate": 3.7973831056378686e-05, "epoch": 2.405233788724264, "s

{"loss": 1.3115901364088058, "learning_rate": 3.484702713103714e-05, "epoch": 3.030594573792573, "step": 31500}
{"loss": 1.2710634810626507, "learning_rate": 3.460650375216471e-05, "epoch": 3.078699249567058, "step": 32000}
{"loss": 1.2850368425250054, "learning_rate": 3.4365980373292286e-05, "epoch": 3.126803925341543, "step": 32500}
{"loss": 1.2339506326913834, "learning_rate": 3.4125456994419855e-05, "epoch": 3.1749086011160284, "step": 33000}
{"loss": 1.2858452498614787, "learning_rate": 3.388493361554743e-05, "epoch": 3.223013276890514, "step": 33500}
{"loss": 1.3019303193688392, "learning_rate": 3.3644410236675005e-05, "epoch": 3.271117952664999, "step": 34000}
{"loss": 1.287760569036007, "learning_rate": 3.340388685780258e-05, "epoch": 3.3192226284394843, "step": 34500}
{"loss": 1.3079655755162238, "learning_rate": 3.3163363478930156e-05, "epoch": 3.3673273042139695, "step": 35000}
{"loss": 1.2996934065818786, "learning_rate": 3.292284010005773e-05, "epoch": 3.4154319799884547, 

...

{"loss": 1.1670540046989917, "learning_rate": 1.99345776409467e-05, "epoch": 6.01308447181066, "step": 62500}
{"loss": 1.0899380594193935, "learning_rate": 1.9694054262074273e-05, "epoch": 6.061189147585146, "step": 63000}
{"loss": 1.118738705933094, "learning_rate": 1.945353088320185e-05, "epoch": 6.10929382335963, "step": 63500}
{"loss": 1.1099229986965655, "learning_rate": 1.9213007504329424e-05, "epoch": 6.157398499134116, "step": 64000}
{"loss": 1.1373853596746921, "learning_rate": 1.8972484125456995e-05, "epoch": 6.205503174908601, "step": 64500}
{"loss": 1.1209191954433917, "learning_rate": 1.8731960746584567e-05, "epoch": 6.253607850683086, "step": 65000}
{"loss": 1.103322835803032, "learning_rate": 1.8491437367712142e-05, "epoch": 6.301712526457572, "step": 65500}
{"loss": 1.115586416900158, "learning_rate": 1.8250913988839714e-05, "epoch": 6.349817202232057, "step": 66000}
{"loss": 1.1374007642269135, "learning_rate": 1.801039060996729e-05, "epoch": 6.397921878006542, "step":

{"loss": 1.1122530838549136, "learning_rate": 1.4883586684625748e-05, "epoch": 7.023282663074851, "step": 73000}
{"loss": 1.077769923567772, "learning_rate": 1.4643063305753321e-05, "epoch": 7.071387338849336, "step": 73500}
{"loss": 1.0941560049057006, "learning_rate": 1.4402539926880893e-05, "epoch": 7.119492014623821, "step": 74000}
{"loss": 1.0665165768563747, "learning_rate": 1.4162016548008467e-05, "epoch": 7.167596690398307, "step": 74500}
{"loss": 1.0776900710761548, "learning_rate": 1.392149316913604e-05, "epoch": 7.215701366172792, "step": 75000}
{"loss": 1.0986787087917327, "learning_rate": 1.3680969790263615e-05, "epoch": 7.263806041947277, "step": 75500}
{"loss": 1.0703114159107208, "learning_rate": 1.3440446411391189e-05, "epoch": 7.311910717721762, "step": 76000}
{"loss": 1.084713762640953, "learning_rate": 1.3199923032518762e-05, "epoch": 7.360015393496248, "step": 76500}
{"loss": 1.07640082898736, "learning_rate": 1.2959399653646334e-05, "epoch": 7.408120069270733, "st

{"loss": 1.0251803379058837, "learning_rate": 9.83259572830479e-06, "epoch": 8.033480854339041, "step": 83500}
{"loss": 1.044325642377138, "learning_rate": 9.592072349432366e-06, "epoch": 8.081585530113527, "step": 84000}
{"loss": 1.0317953082323075, "learning_rate": 9.351548970559938e-06, "epoch": 8.129690205888012, "step": 84500}
{"loss": 1.0015536547601223, "learning_rate": 9.111025591687513e-06, "epoch": 8.177794881662498, "step": 85000}
{"loss": 1.0592361744642258, "learning_rate": 8.870502212815086e-06, "epoch": 8.225899557436982, "step": 85500}
{"loss": 1.0481411541104317, "learning_rate": 8.62997883394266e-06, "epoch": 8.274004233211468, "step": 86000}
{"loss": 1.0429347530901432, "learning_rate": 8.389455455070233e-06, "epoch": 8.322108908985953, "step": 86500}
{"loss": 1.0359599581360817, "learning_rate": 8.148932076197807e-06, "epoch": 8.370213584760439, "step": 87000}
{"loss": 1.0615962873995304, "learning_rate": 7.90840869732538e-06, "epoch": 8.418318260534924, "step": 875

{"loss": 1.0173598893284799, "learning_rate": 4.781604771983837e-06, "epoch": 9.043679045603232, "step": 94000}
{"loss": 1.0090839240849019, "learning_rate": 4.5410813931114104e-06, "epoch": 9.091783721377718, "step": 94500}
{"loss": 0.9924930114448071, "learning_rate": 4.300558014238984e-06, "epoch": 9.139888397152204, "step": 95000}
{"loss": 1.018649611711502, "learning_rate": 4.060034635366558e-06, "epoch": 9.18799307292669, "step": 95500}
{"loss": 1.0094648390710355, "learning_rate": 3.819511256494132e-06, "epoch": 9.236097748701173, "step": 96000}
{"loss": 1.011359853297472, "learning_rate": 3.5789878776217047e-06, "epoch": 9.284202424475659, "step": 96500}
{"loss": 1.0041653116345406, "learning_rate": 3.338464498749279e-06, "epoch": 9.332307100250144, "step": 97000}
{"loss": 1.0207116776704788, "learning_rate": 3.097941119876852e-06, "epoch": 9.38041177602463, "step": 97500}
{"loss": 1.0138043597340585, "learning_rate": 2.8574177410044256e-06, "epoch": 9.428516451799116, "step": 

TrainOutput(global_step=103940, training_loss=1.2277424066718357)

In [0]:
model.save_pretrained('./GPT2new-2')

In [0]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2TokenizerFast
import torch

config = GPT2Config()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', max_len=512)
tokenizer.add_tokens('<pad>')
tokenizer.pad_token = '<pad>'

model = GPT2LMHeadModel.from_pretrained('./GPT2new-2/checkpoint-100000', config=config)

In [0]:
modelnew = GPT2LMHeadModel.from_pretrained('./GPT2new-2/checkpoint-100000', config=config)

In [13]:
modelbase = GPT2LMHeadModel.from_pretrained('gpt2')

In [30]:
prompt = '''JOHNNY
Well because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.

MARK
So, I mean, what's the interesting part?

JOHNNY
Well the interesting part is that
'''
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
outputs = modelnew.generate(inputs, max_length=300, do_sample=True, top_p=0.95, top_k=100, temperature=1.1)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [31]:
tokenizer.decode(outputs[0].numpy())

'JOHNNY\nWell because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.\n\nMARK\nSo, I mean, what\'s the interesting part?\n\nJOHNNY\nWell the interesting part is that\n.\', just! I hate. It went-- of! eyebrow.you! What good! Daisy! Do to" was! What he!my! You was! What I am! I was a first! I said! If took. You have! I\'m was got came! It\'re is would asGasia. It left met, at for haven! You said! It was trying haven! It wrote. You was -of got both wanted! I ever This can took a for guy fucked the ever, in ever got got so or got brought a... Be littleumi, to Can got the-- You didn just a bastard, if missed trying fucked, in all much! You made high bitch a state. Who with no, if won find a rule meeting got... Be a lot a nice sweet big-- any reason much! You told Over a lot a great little child enough idiot more great gift. How or own team a good child for s

In [0]:
with open("Room-new.txt", "w") as f:
  f.write(tokenizer.decode(outputs[0].numpy()))

In [34]:
outputs = modelbase.generate(inputs, max_length=300, do_sample=True, top_p=0.95, top_k=60, temperature=1.1)
tokenizer.decode(outputs[0].numpy())

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


"JOHNNY\nWell because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.\n\nMARK\nSo, I mean, what's the interesting part?\n\nJOHNNY\nWell the interesting part is that\nthere was nothing to do as you were in the train, just an old man's bed, and she looked at me that way. I didn't even think that about, because of her. That was one of the reasons why my heart began beating. That was just like a dream for me. But when you are sitting through the train, when it comes to waking up—I just think it's funny to do it this way.\n\nMARK\nWhat did you get into when you left home. Did you talk to him about the incident?\n\nJOHNNY\nWell, I think about this too much. I was going to buy you a house. I didn't pay for it, because it was such a bad thing to me to come to Mexico, to be such a rich man who was running a little business down here.\n\nMARK\nAll right, good! Ju

In [0]:
with open("Room-base.txt", "w") as f:
  f.write(tokenizer.decode(outputs[0].numpy()))

In [0]:
model2short = GPT2LMHeadModel.from_pretrained('./GPT2-2/checkpoint-10000')

In [39]:
outputs = model2short.generate(inputs, max_length=300, do_sample=True, top_p=0.95, top_k=60, temperature=1.1)
tokenizer.decode(outputs[0].numpy())

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


"JOHNNY\nWell because it was an out of state bank. Anyway, I was working as a busboy in a hotel, and she was sitting, drinking her coffee, and she was so beautiful, and I say hi to her. That’s how we met.\n\nMARK\nSo, I mean, what's the interesting part?\n\nJOHNNY\nWell the interesting part is that\nyou don't wanna know. It was an out of state and I wasn\x92t in there. This wasn\x92t the day that I lived a life with my dad. It\x92s nice to have a second family but it\x92s only one I like. But that\x92s a long story. At one point in the past, I\x92m just getting off the boat, and I\x92m like the kid, but I'm just here, doing the business for this mom.I\x92m, uh...I\x92d think this is it, right? Is that\x92t it? It's a...you know, I\x92re not gonna lie. You just got to--I mean this isn\x92t a--that\x92s not the same? You don\x92t, it\x92s fine. I\x92m good, I\x92m all so much, but...I just said, you\x92re doing that for a chance I\x92d take a"

In [0]:
with open("Room-2short.txt", "w") as f:
  f.write(tokenizer.decode(outputs[0].numpy()))